FRI-based PCS

## How FRI works
A FRI protocol is a protocol for proving that a function $f : H \to \mathbb{F}$ is closed to a polynomial of low degree $d$, where $d \ll |H|$. 
The FRI PCS outputs the root of the Merkle tree as the commitment to the function $f$ while the FRI protocol consists of the evaluation proof.
The protocol can be divided into two phases: commit and query.
In the commit phash, the prover commits to (via Merkle trees) a series of functions generated from $f$ and random elements $v_0, v_1, ... $ from $\mathbb{K}$ provided by the verifier at each round. 
Then in the query phase, the prover provides a set of evaluations of the previously committed functions at a point randomly chosen by the verifier.

### Commit Phase
Denote by $p_0$ the function of low degree. 
In the commit phase, the polynomial $p_0$ is split into two polynomials $g_{0,1}, g_{0,2} : H^2 \to \mathbb{K}$ of degree lower than $d/2$ such that:
$$p_0(X) = g_{0,1}(X^2) + X g_{0,2}(X^2).$$

Then, the verifier sends a uniform randomness $v_0 \in \mathbb{K}$ and ask the prover to commit to the polynomial: $p_1(X) = g_{0,1}(X) + v_0 g_{0,2}(X)$.
Note that $p_1$ is a polynomial of degree less than $d/2$ and the commitment of $p_1$ is not over $H$ but over $H^2 = \{x^2 : x \in H \}$.

The prover then continues by splitting $p_1$ into $g_{1,1}$ and $g_{1,2}$ of degree lower than $d/4$, and constructs $p_2$ with a uniformly sampled $v_1 \in \mathbf{K}$ sent by the verifier.

This above interaction iterates $k = \log d$ times.
At last, $deg(p_k) = 0$ and the prover sends a constant $p_k$, representing a polynomial of degree lower than 1, to the verifier. 

### Query Phase
In the query phase, the verifier sends a randomness $r \in H$ to the prover and queries the evaluations $p_0(r), p_0(-r)$ and $p_1(r_2)$.
From $p_0(r), p_0(-r)$ the verifier computes $p_1(r^2)$ and checks that the computed value matches $p_1(r^2)$ sent by the prover.
In detail, $p_1(r^2)$ can be obtained as follows:
- $p_0(r) = g_{0,1}(r^2) + r\cdot g_{0,2}(r^2)$;
- $p_0(-r) = g_{0,1}(r^2) - r\cdot g_{0,2}(r^2)$;
- $p_1(r^2) = g_{0,1}(r^2) + v_0\cdot g_{0,2}(r^2)$.

In the next interaction, the verifier queries $p_1(-r^2)$ and $p_2(r^4)$ and checks the consistency between $p_1, p_2$ as before.
The interaction repeats till the constant $p_k$. 
The verifier checks that the value sent by the prover is indeed equal to the value that the verifier computed from the queries up until $p_{k-1}$.
To fully ensure correctness, the prover must accompany the evaluations that she sends with a claim of their existence (via Merkle tree paths).
Upon the completion of this process, the verifier has a first confirmation that the polynomials committed in the commit phase $p_0, p_1, . . . , p_k$ are consistent with each other.

Finally, to achieve the required bounds for the soundness of the protocol, the query phase is repeated multiple times.

## Codes
There are 8 files in Plonky3/fri/src: config.rs, fold_even_odd.rs, hiding_pcs.rs, proof.rs, prover.rs, two_adic_pcs, and verifier.rs.

### config.rs
There are two main functions in this code: FriConfig and FriGenericConfig.
FriConfig defines the configuration parameters for the FRI protocol, including the blow-up factor (log_blowup), the number of queries(num_queries), and anti-attack parameters(proof_of_work_bits).
FriGenericConfig provides an abstract interface for specific implementations of FRI, including fold_row and fold_matrix for folding opreation.

### fold_even_odd.rs
This code implements a folding operation the fold_even_odd function:
$p(x) = p_{even}(x^2) + x p_{odd}(x^2)$

### hiding_pcs.rs
This code implements a hiding pcs by adding random codewords.

### proof.rs
combine communications into proofs.
1. FriProof
2. QueryProof
3. CommitPhaseProofStep

### Prover.rs
function Prove.
main stage: commit -> challenge -> query ->pack proofs

commit
answer_query

### two_adic_pcs.rs
FRI over two-adic field.

1. define sturct TwoAdicFriPcs
2. a new TwoAdicFriPcs instance
---------def-----------
3. Generic config
4. fold_row
5. fold_matrix  
--------config----------
6. interface for TwoAdicFriPcs
    - natural_domain_for_degree
    - commit

### verifier.rs
function verify verify_query